<a href="https://colab.research.google.com/github/Daniel-chang111/Quant-trading-with-Deepseek/blob/main/AI_KOSPI500_By_Claude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# ============================================================
# 🚀 코스피 종목 데이터 수집 + AI 학습 시스템 (버그 수정판)
# ============================================================

!pip install pykrx pandas numpy scikit-learn -q

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from pykrx import stock
import time, os, pickle, warnings
from concurrent.futures import ThreadPoolExecutor, as_completed
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')


# ============================================================
# 캐시 시스템
# ============================================================
class StockDataCache:
    def __init__(self, cache_dir='stock_cache'):
        self.cache_dir = cache_dir
        os.makedirs(cache_dir, exist_ok=True)

    def get_cache_path(self, ticker, date):
        return f"{self.cache_dir}/{ticker}_{date}.pkl"

    def save(self, ticker, date, data):
        with open(self.get_cache_path(ticker, date), 'wb') as f:
            pickle.dump(data, f)

    def load(self, ticker, date):
        path = self.get_cache_path(ticker, date)
        if os.path.exists(path):
            with open(path, 'rb') as f:
                return pickle.load(f)
        return None

cache = StockDataCache()


# ============================================================
# ✅ 수정 1: 유효 거래일 탐색 함수 (2025년 수집에도 적용)
# ============================================================
def find_valid_date(target_date, max_days_back=5):
    """실제 거래일 찾기 - 앞뒤로 탐색"""
    date_obj = datetime.strptime(target_date, '%Y%m%d')
    for i in range(max_days_back + 1):
        check_date = (date_obj - timedelta(days=i)).strftime('%Y%m%d')
        try:
            tickers = stock.get_market_ticker_list(check_date, market="KOSPI")
            if tickers and len(tickers) > 0:
                if i > 0:
                    print(f"  📅 {target_date} → {check_date}로 보정 ({len(tickers)}개 종목)")
                return check_date
        except:
            continue
    return None


# ============================================================
# ✅ 수정 2: 단일 종목 수집 (컬럼명 안전 접근 + 딜레이)
# ============================================================
def fetch_ticker_data(ticker, date):
    cached = cache.load(ticker, date)
    if cached is not None:  # ✅ 수정: `if cached`는 빈 dict도 False 처리됨
        return cached

    try:
        name = stock.get_market_ticker_name(ticker)

        df_fund = stock.get_market_fundamental(date, date, ticker)
        if df_fund is None or df_fund.empty:
            return None

        fund_row = df_fund.iloc[0]

        # ✅ 수정: 실제 컬럼명 확인 후 접근
        cols = fund_row.index.tolist()
        per = fund_row['PER'] if 'PER' in cols else np.nan
        pbr = fund_row['PBR'] if 'PBR' in cols else np.nan

        if pd.isna(per) or pd.isna(pbr) or per <= 0 or pbr <= 0:
            return None

        start_date = (datetime.strptime(date, '%Y%m%d') - timedelta(days=200)).strftime('%Y%m%d')
        df_price = stock.get_market_ohlcv_by_date(start_date, date, ticker)

        if df_price is None or len(df_price) < 100:
            return None

        closes = df_price['종가']
        momentum_1m = (closes.iloc[-1] / closes.iloc[-22] - 1) * 100
        momentum_3m = (closes.iloc[-1] / closes.iloc[-66] - 1) * 100
        momentum_6m = (closes.iloc[-1] / closes.iloc[-132] - 1) * 100 if len(df_price) > 132 else 0
        volatility = df_price['등락률'].tail(60).std()

        result = {
            '티커': ticker, '종목': name,
            'PER': round(per, 2), 'PBR': round(pbr, 2),
            '모멘텀_1m': round(momentum_1m, 2),
            '모멘텀_3m': round(momentum_3m, 2),
            '모멘텀_6m': round(momentum_6m, 2),
            '변동성': round(volatility, 2),
            '현재가': closes.iloc[-1]
        }
        cache.save(ticker, date, result)
        return result

    except Exception as e:
        return None


# ============================================================
# ✅ 수정 3: 병렬 처리 - workers 축소 + 딜레이 추가
# ============================================================
def collect_month_data(date, max_workers=3):  # ✅ 10 → 3으로 변경
    print(f"📅 {date} 수집 시작...")

    try:
        all_tickers = stock.get_market_ticker_list(date, market="KOSPI")
        if not all_tickers or len(all_tickers) == 0:
            print(f"  ⚠️ {date}: 종목 없음 (휴장일 가능성)")
            return pd.DataFrame()
        tickers = all_tickers[:500]
        print(f"  📋 수집 대상: {len(tickers)}개")
    except Exception as e:
        print(f"  ❌ 종목 리스트 조회 실패: {e}")
        return pd.DataFrame()

    results = []
    success_count = 0

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(fetch_ticker_data, ticker, date): ticker
                   for ticker in tickers}
        for i, future in enumerate(as_completed(futures)):
            result = future.result()
            if result:
                result['기준일'] = date
                results.append(result)
                success_count += 1

            # ✅ 수정: 50개마다 딜레이로 Rate Limit 방지
            if i % 50 == 49:
                time.sleep(1)

    df = pd.DataFrame(results)
    print(f"  ✅ 성공: {success_count}개 / 전체: {len(tickers)}개")
    return df


# ============================================================
# ✅ 수정 4: 2025년 수집 루프에 find_valid_date() 적용
# ============================================================
def collect_2025_full_data():
    # 월말 기준 날짜 (거래일 아닐 수 있으므로 find_valid_date로 보정)
    month_targets = [
        '20250131', '20250228', '20250331', '20250430',
        '20250530', '20250630', '20250731', '20250829',
        '20250930', '20251031', '20251128', '20251230'
    ]

    print("📊 2025년 실제 거래일 확인중...")
    valid_months = []
    for m in month_targets:
        valid = find_valid_date(m)  # ✅ 핵심 수정: 여기 적용!
        if valid:
            valid_months.append(valid)
        else:
            print(f"  ❌ {m}: 유효한 거래일 없음")

    print(f"\n✅ 수집할 월: {valid_months}\n")

    all_data = []
    start_time = time.time()

    for i, month in enumerate(valid_months, 1):
        print(f"\n[{i}/{len(valid_months)}] ", end="")
        df_month = collect_month_data(month)

        if df_month is not None and len(df_month) > 0:
            all_data.append(df_month)

        # ✅ 수정: 월간 요청 사이 2초 딜레이
        time.sleep(2)
        elapsed = time.time() - start_time
        remaining = (elapsed / i) * (len(valid_months) - i)
        print(f"  ⏱️ 경과: {elapsed:.0f}초 / 잔여: {remaining:.0f}초")

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        print(f"\n✅ 수집 완료: {len(final_df)}개 샘플, {final_df['기준일'].nunique()}개월")
        return final_df
    else:
        print("❌ 수집 실패: 모든 월에서 데이터 없음")
        print("💡 pykrx 버전 확인: pip show pykrx")
        print("💡 직접 테스트: stock.get_market_ticker_list('20250131', market='KOSPI')")
        return pd.DataFrame()


# ============================================================
# 다음달 수익률 추가
# ============================================================
def add_future_returns_full(df):
    if df.empty:
        return pd.DataFrame()

    results = []
    for ticker, group in df.groupby('티커'):
        group = group.sort_values('기준일').reset_index(drop=True)
        if len(group) < 2:
            continue
        for i in range(len(group) - 1):
            row = group.iloc[i].to_dict()
            next_price = group.iloc[i + 1]['현재가']
            ret = (next_price / row['현재가'] - 1) * 100
            row['다음달수익률'] = round(ret, 2)
            row['target'] = 1 if ret > 0 else 0
            results.append(row)

    df_result = pd.DataFrame(results)
    print(f"✅ 학습 샘플: {len(df_result)}개 / 종목 수: {df_result['티커'].nunique()}개")
    return df_result


# ============================================================
# AI 모델 학습
# ============================================================
def train_ai_model_full(df_train):
    feature_cols = ['PER', 'PBR', '모멘텀_1m', '모멘텀_3m', '변동성']
    df_train = df_train.dropna(subset=feature_cols)

    X = df_train[feature_cols].values
    y = df_train['target'].values

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_val, y_train, y_val = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42, stratify=y)

    model = RandomForestClassifier(
        n_estimators=200, max_depth=10,
        min_samples_split=5, random_state=42, n_jobs=-1)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    print(f"✅ 검증 정확도: {acc:.2%}")
    print(classification_report(y_val, y_pred, target_names=['하락', '상승']))
    return model, scaler, feature_cols


# ============================================================
# 2026년 1월 종목 선정 & 2월 성과 확인 (기존과 동일)
# ============================================================
def select_stocks_jan2026(model, scaler, features, date='20260130'):
    valid_date = find_valid_date(date)
    if not valid_date:
        print("❌ 유효한 거래일 없음")
        return pd.DataFrame()

    tickers = stock.get_market_ticker_list(valid_date, market="KOSPI")[:500]
    stock_data = []

    for ticker in tickers:
        try:
            name = stock.get_market_ticker_name(ticker)
            df_fund = stock.get_market_fundamental(valid_date, valid_date, ticker)
            if df_fund is None or df_fund.empty:
                continue

            fund_row = df_fund.iloc[0]
            per = fund_row.get('PER', np.nan)
            pbr = fund_row.get('PBR', np.nan)

            if pd.isna(per) or pd.isna(pbr) or per <= 0:
                continue

            start_date = (datetime.strptime(valid_date, '%Y%m%d') - timedelta(days=200)).strftime('%Y%m%d')
            df_price = stock.get_market_ohlcv_by_date(start_date, valid_date, ticker)
            if df_price is None or len(df_price) < 100:
                continue

            closes = df_price['종가']
            stock_data.append({
                '종목': name, '티커': ticker,
                'PER': per, 'PBR': pbr,
                '모멘텀_1m': (closes.iloc[-1] / closes.iloc[-22] - 1) * 100,
                '모멘텀_3m': (closes.iloc[-1] / closes.iloc[-66] - 1) * 100,
                '변동성': df_price['등락률'].tail(60).std(),
                '1월말_가격': closes.iloc[-1]
            })
        except:
            continue

    df_jan = pd.DataFrame(stock_data)
    if df_jan.empty:
        return pd.DataFrame()

    X_jan = scaler.transform(df_jan[features].values)
    df_jan['AI_점수'] = model.predict_proba(X_jan)[:, 1]
    top_20 = df_jan.nlargest(20, 'AI_점수')
    print(top_20[['종목', 'PER', 'PBR', '모멘텀_1m', 'AI_점수']].to_string(index=False))
    return top_20


def check_feb_results(portfolio_jan, date='20260213'):
    results = []
    for _, row in portfolio_jan.iterrows():
        try:
            df_feb = stock.get_market_ohlcv_by_date(date, date, row['티커'])
            if df_feb.empty:
                continue
            ret = (df_feb['종가'].iloc[0] / row['1월말_가격'] - 1) * 100
            results.append({'종목': row['종목'], '수익률(%)': round(ret, 2)})
        except:
            continue

    df_res = pd.DataFrame(results).sort_values('수익률(%)', ascending=False)
    print(df_res.to_string(index=False))
    print(f"\n평균 수익률: {df_res['수익률(%)'].mean():.2f}%")
    print(f"승률: {len(df_res[df_res['수익률(%)']>0])/len(df_res)*100:.1f}%")
    return df_res


# ============================================================
# 메인 실행
# ============================================================
df_2025_full = collect_2025_full_data()

if not df_2025_full.empty:
    df_train = add_future_returns_full(df_2025_full)
    if not df_train.empty:
        model, scaler, features = train_ai_model_full(df_train)
        portfolio_jan = select_stocks_jan2026(model, scaler, features)
        if not portfolio_jan.empty:
            feb_results = check_feb_results(portfolio_jan)

📊 2025년 실제 거래일 확인중...

✅ 수집할 월: ['20250131', '20250228', '20250331', '20250430', '20250530', '20250630', '20250731', '20250829', '20250930', '20251031', '20251128', '20251230']


[1/12] 📅 20250131 수집 시작...
  📋 수집 대상: 500개
  ✅ 성공: 311개 / 전체: 500개
  ⏱️ 경과: 21초 / 잔여: 229초

[2/12] 📅 20250228 수집 시작...
  📋 수집 대상: 500개
  ✅ 성공: 310개 / 전체: 500개
  ⏱️ 경과: 63초 / 잔여: 314초

[3/12] 📅 20250331 수집 시작...
  📋 수집 대상: 500개
  ✅ 성공: 310개 / 전체: 500개
  ⏱️ 경과: 121초 / 잔여: 364초

[4/12] 📅 20250430 수집 시작...
  📋 수집 대상: 500개
  ✅ 성공: 313개 / 전체: 500개
  ⏱️ 경과: 179초 / 잔여: 358초

[5/12] 📅 20250530 수집 시작...
  📋 수집 대상: 500개
  ✅ 성공: 326개 / 전체: 500개
  ⏱️ 경과: 236초 / 잔여: 331초

[6/12] 📅 20250630 수집 시작...
  📋 수집 대상: 500개
  ✅ 성공: 326개 / 전체: 500개
  ⏱️ 경과: 293초 / 잔여: 293초

[7/12] 📅 20250731 수집 시작...
  📋 수집 대상: 500개
  ✅ 성공: 326개 / 전체: 500개
  ⏱️ 경과: 350초 / 잔여: 250초

[8/12] 📅 20250829 수집 시작...
  📋 수집 대상: 500개
  ✅ 성공: 327개 / 전체: 500개
  ⏱️ 경과: 406초 / 잔여: 203초

[9/12] 📅 20250930 수집 시작...
  📋 수집 대상: 500개
  ✅ 성공: 327개 / 전체: 500개
  ⏱️ 경과: 462